In [ ]:
def extract_trace_group(
    dat: ExSum, group: str = "dF", trace: str = "matchFilt", extract_chan: int = 2
):
    n_chunks = len(dat.data["E"])
    full_data = {}
    full_data["DMD1"] = []
    full_data["DMD2"] = []
    for dmd in [1, 2]:
        for chunk in range(n_chunks):
            if dat.data["E"][chunk][dmd - 1][group][trace].ndim == 3:
                full_data[f"DMD{dmd}"].append(
                    dat.data["E"][chunk][dmd - 1][group][trace][:, :, extract_chan - 1]
                )
            elif dat.data["E"][chunk][dmd - 1][group][trace].ndim == 2:
                full_data[f"DMD{dmd}"].append(
                    dat.data["E"][chunk][dmd - 1][group][trace]
                )
            else:
                raise ValueError("Unexpected number of dimensions")
        full_data[f"DMD{dmd}"] = np.concatenate(full_data[f"DMD{dmd}"], axis=1)
    return full_data


In [ ]:
def _generate_frame_differences(video_path):

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise OSError(f"Could not open video: {video_path}")

    diffs = []
    prev_gray = None
    try:
        while True:
            ok, frame = cap.read()
            if not ok or frame is None:
                break

            if frame.ndim == 3:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            else:
                gray = frame

            gray = gray.astype(np.float32, copy=False)

            if prev_gray is None:
                diffs.append(0.0)
            else:
                diff_val = np.mean(np.abs(gray - prev_gray))
                print('appending diff')
                diffs.append(float(diff_val))

            prev_gray = gray
    finally:
        cap.release()

    return np.array(diffs, dtype=float)

In [1]:
%reload_ext autoreload
%autoreload 2
import slap2_py as spy
import wisco_slap as wis
import wisco_slap.defs as DEFS
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import matplotlib as mpl
import h5py
import glob
import polars as pl
from pathlib import Path
import tifffile
import seaborn as sns
import electro_py as epy
import cv2

In [2]:
subject = 'avior'
exp = 'exp_2'
sb=1
loc = 'loc_K'
acq = 'acq_2'

In [3]:
wis.pipes.traces.save_activity_dataframes(subject, exp, loc, acq, overwrite=False)

ERROR:root:ERROR: MATLAB type not supported: string, (uint32)


trl 30 is DEAD on DMD 1, appending nan array
trl 33 is DEAD on DMD 1, appending nan array
trl 107 is DEAD on DMD 1, appending nan array
trl 190 is DEAD on DMD 1, appending nan array
trl 239 is DEAD on DMD 1, appending nan array
trl 259 is DEAD on DMD 1, appending nan array
trl 30 is DEAD on DMD 1, appending nan array
trl 33 is DEAD on DMD 1, appending nan array
trl 107 is DEAD on DMD 1, appending nan array
trl 190 is DEAD on DMD 1, appending nan array
trl 239 is DEAD on DMD 1, appending nan array
trl 259 is DEAD on DMD 1, appending nan array
trl 30 is DEAD on DMD 1, appending nan array
trl 33 is DEAD on DMD 1, appending nan array
trl 107 is DEAD on DMD 1, appending nan array
trl 190 is DEAD on DMD 1, appending nan array
trl 239 is DEAD on DMD 1, appending nan array
trl 259 is DEAD on DMD 1, appending nan array
trl 30 is DEAD on DMD 1, appending nan array
trl 33 is DEAD on DMD 1, appending nan array
trl 107 is DEAD on DMD 1, appending nan array
trl 190 is DEAD on DMD 1, appending nan ar

In [11]:
eyedf = wis.peri.vid.load_eye_metric_df(subject, exp, sb=1)

In [17]:
soma = wis.scope.io.load_roidf(subject, exp, 'loc_C', 'acq_2', apply_ephys_offset=True)

In [20]:
soma_dfs = []
for loc in locs:
    soma_df = wis.scope.io.load_roidf(subject, exp, loc, acq, apply_ephys_offset=True)
    soma_dfs.append(soma_df)
sdf = pl.concat(soma_dfs)

FileNotFoundError: No such file or directory (os error 2): ...p_mi/slap_mi/analysis_materials/avior/exp_2/activity_data/loc_K/acq_2/roidf_Fsvd.parquet (set POLARS_VERBOSE=1 to see full path)

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


In [13]:
sdnp.shape

(417924,)